In [1]:
import datetime
import ifcopenshell
from ifcopenshell.api import run
from ifcopenshell.util.element import get_decomposition
from ifcopenshell.util.placement import get_storey_elevation

In [2]:
# Define a convenience function to add a task chained to a predecessor
def add_task(model, name, predecessor, work_schedule):
    # Add a construction task
    task = run("sequence.add_task", model,
        work_schedule=work_schedule, name=name, predefined_type="CONSTRUCTION")

    # Give it a time
    task_time = run("sequence.add_task_time", model, task=task)

    # Arbitrarily set the task's scheduled time duration to be 1 week
    run("sequence.edit_task_time", model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2000, 1, 1), "ScheduleDuration": "P1W"})

    # If a predecessor exists, create a finish to start relationship
    if predecessor:
        run("sequence.assign_sequence", model, relating_process=predecessor, related_process=task)

    return task


In [10]:
# Open an existing IFC4 model you have of a building
model = ifcopenshell.open("data/Duplex_A.ifc")

In [19]:
# Create a new construction schedule
schedule = run("sequence.add_work_schedule", model, name="Construction")

TypeError: in method 'entity_instance_setArgumentAsEntityInstance', argument 3 of type 'IfcUtil::IfcBaseClass *'

In [20]:
# Let's imagine a starting task for site establishment.
task = add_task(model, "Site establishment", None, schedule)
start_task = task

NameError: name 'schedule' is not defined

In [21]:
# Get all our storeys sorted by elevation ascending.
storeys = sorted(model.by_type("IfcBuildingStorey"), key=lambda s: get_storey_elevation(s))

In [22]:
print(model.by_type("IfcBuildingStorey"))

[#39=IfcBuildingStorey('1xS3BCk291UvhgP2dvNMKI',#33,'Level 1',$,$,#38,$,$,.ELEMENT.,0.), #43=IfcBuildingStorey('1xS3BCk291UvhgP2dvNMQJ',#33,'Level 2',$,$,#42,$,$,.ELEMENT.,3.100000000000378), #47=IfcBuildingStorey('1xS3BCk291UvhgP2dvNsgp',#33,'T/FDN',$,$,#46,$,$,.ELEMENT.,-1.25), #51=IfcBuildingStorey('1xS3BCk291UvhgP2dvNtSE',#33,'Roof',$,$,#50,$,$,.ELEMENT.,6.00000000000039)]


In [23]:
print(len(storeys))

4


In [24]:
# For each storey ...
for storey in storeys:

    # Add a construction task to construct that storey, using our convenience function
    task = add_task(model, f"Construct {storey.Name}", task, schedule)

    # Assign all the products in that storey to the task as construction outputs.
    for product in get_decomposition(storey):
        run("sequence.assign_product", model, relating_product=product, related_object=task)

NameError: name 'task' is not defined

In [25]:
# Ask the computer to calculate all the dates for us from the start task.
# For example, if the first task started on the 1st of January and took a
# week, the next task will start on the 8th of January. This saves us
# manually doing date calculations.
run("sequence.cascade_schedule", model, task=start_task)

NameError: name 'start_task' is not defined

In [27]:
# Calculate the critical path and floats.
run("sequence.recalculate_schedule", model, work_schedule=schedule)

# Write out to a file
model.write("data/Duplex_A_Scedule.ifc")

NameError: name 'schedule' is not defined